# Creación de target y FE

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
%%bash
pip install duckdb
pip install jupysql
pip install duckdb-engine
pip install optuna==3.6.1

In [ ]:
import duckdb
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

import lightgbm as lgb

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from time import time

import pickle


%reload_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
#%config SqlMagic.style = "SINGLE_BORDER"
#%config SqlMagic.style = "PLAIN_COLUMNS"
%sql duckdb://

In [ ]:
dataset_path = '/content/drive/MyDrive/EyF/'
dataset_file = 'competencia_01_crudo.csv'

In [ ]:
%%sql
create or replace table competencia_01 as
with base as (
    select *
    from read_csv_auto("{{dataset_path + dataset_file}}")
),maximos as (
    select max(foto_mes) as max_foto_mes
    from base
),
-- join con futuro +1
join1 as (
    select b.numero_de_cliente, b.foto_mes
    from base b
    join base f
      on b.numero_de_cliente = f.numero_de_cliente
     and f.foto_mes = b.foto_mes + 1
),
-- join con futuro +2
join2 as (
    select b.numero_de_cliente, b.foto_mes
    from base b
    join base f
      on b.numero_de_cliente = f.numero_de_cliente
     and f.foto_mes = b.foto_mes + 2
)
select
    b.*,
    case
        -- caso 1: último mes -> NULL
        when b.foto_mes = m.max_foto_mes then null

        -- caso 2: penúltimo mes
        when b.foto_mes = m.max_foto_mes - 1
            then case
                when j1.numero_de_cliente is not null then null     -- hay foto_mes+1 => no se puede calcular
                else 'BAJA+1'                       -- no esta en foto_mes+1
            end

        -- resto de meses
        when j2.numero_de_cliente is not null then 'CONTINUA'
        when j1.numero_de_cliente is not null then 'BAJA+2'
        else 'BAJA+1'
    end as clase_ternaria
from base b
cross join maximos m
left join join1 j1
  on b.numero_de_cliente = j1.numero_de_cliente and b.foto_mes = j1.foto_mes
left join join2 j2
  on b.numero_de_cliente = j2.numero_de_cliente and b.foto_mes = j2.foto_mes;



In [ ]:
%%sql
create or replace table competencia_01 as
select
    *,
    ifnull(ctarjeta_visa_transacciones, 0) + ifnull(ctarjeta_master_transacciones, 0) + ifnull(ctarjeta_debito_transacciones, 0) as new_cantransacciones
from competencia_01;

In [ ]:
%%sql
create or replace table competencia_01 as
select
    *,
    ifnull(mtarjeta_master_consumo, 0) + ifnull(mtarjeta_visa_consumo, 0) + ifnull(mautoservicio, 0) as new_montransacciones
from competencia_01;

In [ ]:
%%sql
create or replace table competencia_01 as
select
    *,
    case
        when mcuentas_saldo <= 0 then 1
        else 0
    end as new_deudor
from competencia_01;

In [ ]:
%%sql
create or replace table competencia_01 as
select
    *,
    ifnull(mpayroll, 0) + ifnull(mpayroll2, 0) as new_totalpayroll
  from competencia_01;

In [ ]:
%%sql
create or replace table competencia_01 as
select
    *,
    ifnull(Master_mlimitecompra, 0) + ifnull(Visa_mlimitecompra, 0) as new_limitedecompra
  from competencia_01;

In [ ]:
%%sql
create or replace table competencia_01 as
select
    *,
    ifnull(Master_delinquency, 0) + ifnull(Visa_delinquency, 0) as new_alldelincuecy
  from competencia_01;

In [ ]:
%%sql
create or replace table competencia_01 as
select
    *,
    ifnull(mcomisiones, 0) + ifnull(mactivos_margen, 0) + ifnull(mpasivos_margen, 0) as new_total_gain
  from competencia_01;

In [ ]:
%%sql
create or replace table competencia_01 as
select
    *,
    ifnull(mcajeros_propios_descuentos, 0) + ifnull(mtarjeta_visa_descuentos, 0) + ifnull(mtarjeta_master_descuentos, 0) as new_total_discounts
  from competencia_01;

In [ ]:
%%sql
create or replace table competencia_01 as
select
    *,
    ifnull(mcomisiones_mantenimiento, 0) + ifnull(mcomisiones_otras, 0) as new_total_comiss
  from competencia_01;

In [ ]:
%%sql
create or replace table competencia_01 as
select
    *,
    ifnull(mcuenta_debitos_automaticos, 0) + ifnull(ctarjeta_visa_debitos_automaticos, 0) +  ifnull(ctarjeta_master_debitos_automaticos, 0) +  ifnull(cpagomiscuentas, 0)  +  ifnull(cpagodeservicios, 0) as new_total_debautomaticos
  from competencia_01;

In [ ]:
%%sql
create or replace table competencia_01 as
select
    *,
    (CASE WHEN new_cantransacciones IS NOT NULL AND new_cantransacciones > 0 THEN 1 ELSE 0 END) +
    (CASE WHEN new_totalpayroll IS NOT NULL AND new_totalpayroll > 0 THEN 1 ELSE 0 END) +
    (CASE WHEN Visa_madelantopesos IS NOT NULL AND Visa_madelantopesos > 0 THEN 1 ELSE 0 END) +
    (CASE WHEN Visa_madelantodolares IS NOT NULL AND Visa_madelantodolares > 0 THEN 1 ELSE 0 END) +
    (CASE WHEN Master_madelantopesos IS NOT NULL AND Master_madelantopesos > 0 THEN 1 ELSE 0 END) +
    (CASE WHEN Master_madelantodolares IS NOT NULL AND Master_madelantodolares > 0 THEN 1 ELSE 0 END) +
    (CASE WHEN cmobile_app_trx IS NOT NULL AND cmobile_app_trx > 0 THEN 1 ELSE 0 END) +
    (CASE WHEN ctrx_quarter IS NOT NULL AND ctrx_quarter > 0 THEN 1 ELSE 0 END) +
    (CASE WHEN catm_trx IS NOT NULL AND catm_trx > 0 THEN 1 ELSE 0 END) +
    (CASE WHEN chomebanking_transacciones IS NOT NULL AND chomebanking_transacciones > 0 THEN 1 ELSE 0 END) +
    (CASE WHEN ccheques_emitidos IS NOT NULL AND ccheques_emitidos > 0 THEN 1 ELSE 0 END) +
    (CASE WHEN mcheques_emitidos IS NOT NULL AND mcheques_emitidos > 0 THEN 1 ELSE 0 END) +
    (CASE WHEN cextraccion_autoservicio IS NOT NULL AND cextraccion_autoservicio > 0 THEN 1 ELSE 0 END) +
    (CASE WHEN ctransferencias_emitidas IS NOT NULL AND ctransferencias_emitidas > 0 THEN 1 ELSE 0 END) +
    (CASE WHEN ctransferencias_recibidas IS NOT NULL AND ctransferencias_recibidas > 0 THEN 1 ELSE 0 END) +
    (CASE WHEN cpagodeservicios IS NOT NULL AND cpagodeservicios > 0 THEN 1 ELSE 0 END) +
    (CASE WHEN cpagomiscuentas IS NOT NULL AND cpagomiscuentas > 0 THEN 1 ELSE 0 END) +
    (CASE WHEN active_quarter IS NOT NULL AND active_quarter > 0 THEN 1 ELSE 0 END)
    AS actividad
    from competencia_01;

In [ ]:
%%sql
create or replace table competencia_01 as
select
    *,
    ifnull(mprestamos_personales, 0) + ifnull(mprestamos_prendarios, 0) +  ifnull(mprestamos_hipotecarios, 0) as new_deuda_prestamos,
    ifnull(mplazo_fijo_dolares, 0) + ifnull(mplazo_fijo_pesos, 0) + ifnull(minversion1_pesos, 0) + ifnull(minversion1_dolares, 0) + ifnull(minversion2, 0) as new_total_inversiones,
    ifnull(cseguro_vida, 0) + ifnull(cseguro_auto, 0) + ifnull(cseguro_vivienda, 0) + ifnull(cseguro_accidentes_personales, 0) as new_total_seguro,
  from competencia_01;

In [ ]:
%%sql
create or replace table competencia_01 as
select
    *,
    ifnull(ifnull(new_montransacciones, 0 ) / NULLIF(new_totalpayroll, 0), 0) as ratio_consumopayroll,
    ifnull(ifnull(new_limitedecompra, 0 ) / NULLIF(new_totalpayroll, 0), 0) as ratio_limitepayroll,
    ifnull(ifnull(new_total_comiss, 0) / NULLIF(new_totalpayroll, 0), 0) as ratio_comisspayroll,
    ifnull(ifnull(new_total_gain, 0) / NULLIF(new_totalpayroll, 0), 0) as ratio_gainpayroll,
    ifnull(ifnull(new_montransacciones, 0 ) / NULLIF(mcuentas_saldo, 0), 0) as ratio_consumocuenta,
    ifnull(ifnull(new_limitedecompra, 0 ) / NULLIF(mcuentas_saldo, 0), 0 ) as ratio_limitecuenta,
    ifnull(ifnull(new_total_comiss, 0) / NULLIF(mcuentas_saldo, 0), 0) as ratio_comisscuenta,
    ifnull(ifnull(new_total_gain, 0) / NULLIF(mcuentas_saldo, 0),0) as ratio_gaincuenta,
    ifnull(ifnull(new_montransacciones, 0) / NULLIF(new_cantransacciones, 0),0) as ratio_avrgtransaction,
  from competencia_01;


In [ ]:
%%sql
create or replace table competencia_01 as
select
    *,
    ratio_avrgtransaction * ctrx_quarter as expected_mtrx_quarter,
from competencia_01;

In [ ]:
%%sql
create or replace table competencia_01 as
SELECT
    *,
    AVG(new_totalpayroll) OVER (
        PARTITION BY numero_de_cliente
        ORDER BY foto_mes
        ROWS BETWEEN 3 PRECEDING AND CURRENT ROW
    ) AS mobavr_totalpayroll,
FROM competencia_01
ORDER BY numero_de_cliente, foto_mes;


In [ ]:
%%sql
create or replace table competencia_01 as
SELECT
    *,
    AVG(new_montransacciones) OVER (
        PARTITION BY numero_de_cliente
        ORDER BY foto_mes
        ROWS BETWEEN 3 PRECEDING AND CURRENT ROW
    ) AS mobavr_montransacciones,
FROM competencia_01
ORDER BY numero_de_cliente, foto_mes;


In [ ]:
%%sql
create or replace table competencia_01 as
SELECT
    *,
    AVG(new_cantransacciones) OVER (
        PARTITION BY numero_de_cliente
        ORDER BY foto_mes
        ROWS BETWEEN 3 PRECEDING AND CURRENT ROW
    ) AS mobavr_cantransacciones,
FROM competencia_01
ORDER BY numero_de_cliente, foto_mes;


In [ ]:
%%sql
create or replace table competencia_01 as
SELECT
    *,
    AVG(mrentabilidad) OVER (
        PARTITION BY numero_de_cliente
        ORDER BY foto_mes
        ROWS BETWEEN 3 PRECEDING AND CURRENT ROW
    ) AS mobavr_mrentabilidad,
FROM competencia_01
ORDER BY numero_de_cliente, foto_mes;


In [ ]:
%%sql
create or replace table competencia_01 as
SELECT
    *,
    AVG(mcuentas_saldo) OVER (
        PARTITION BY numero_de_cliente
        ORDER BY foto_mes
        ROWS BETWEEN 3 PRECEDING AND CURRENT ROW
    ) AS mobavr_mcuentas_saldo,
FROM competencia_01
ORDER BY numero_de_cliente, foto_mes;


In [ ]:
%%sql
create or replace table competencia_01 as
SELECT
    *,
    NTILE(10) OVER (ORDER BY mcuentas_saldo) AS mcuentas_saldo
FROM competencia_01;

In [ ]:

%%sql
create or replace table competencia_01 as
SELECT
    *,
    AVG(actividad) OVER (
        PARTITION BY numero_de_cliente
        ORDER BY foto_mes
        ROWS BETWEEN 3 PRECEDING AND CURRENT ROW
    ) AS mobavr_actividad,
FROM competencia_01
ORDER BY numero_de_cliente, foto_mes;

In [ ]:
%%sql
create or replace table competencia_01_lag as
select
    c1.*,
    c2.new_totalpayroll as lag1_new_totalpayroll,
    c2.new_deudor as lag1_new_deudor,
    c2.ratio_avrgtransaction as lag1_ratio_avrgtransaction,
    c2.ratio_consumocuenta as lag1_ratio_consumocuenta,
    c2.new_montransacciones as lag1_new_montransacciones,
    c2.new_cantransacciones as lag1_new_cantransacciones,
    c2.mrentabilidad as lag1_mrentabilidad,
    c2.mcuentas_saldo as lag1_mcuentas_saldo,
    c2.new_limitedecompra as lag1_new_limitedecompra,
    c2.new_total_gain as lag1_new_total_gain,
    c2.new_total_comiss as lag1_new_total_comiss,
    c2.mcomisiones as lag1_mcomisiones,
    c2.new_total_discounts as lag1_new_total_discounts,
    c2.actividad as lag1_actividad,
    c2.mpasivos_margen as lag1_pasivos,
    c2.mactivos_margen as lag1_activos,
    c2.tcuentas as lag1_tcuentas,
    c2.mcuenta_corriente as lag1_mcuenta_corriente,
    c2.mcuenta_corriente_adicional as lag1_mcuentaadicional,
    c2.mcaja_ahorro as lag1_mcajaahorro,
    c2.ctarjeta_debito_transacciones as lag1_cdebitotrans,
    c2.ctarjeta_visa_transacciones as lag1_cisatrans,
    c2.mtarjeta_visa_consumo as lag1_mvisaconsumo,
    c2.ctarjeta_master_transacciones as lag1_cmasterconsumo,
    c2.mtarjeta_master_consumo as lag1_mmasterconsumo,
    c2.mprestamos_personales as lag1_prestamos,
    c2.mplazo_fijo_dolares as lag1_mfijo_dolares,
    c2.mplazo_fijo_pesos as lag1_mfijo_pesos,
    c2.ccuenta_debitos_automaticos as lag1_ccuenta_debitos,
    c2.mcuenta_debitos_automaticos as lag1_mcuenta_debitos,
    c2.ctarjeta_visa_debitos_automaticos as lag1_visa_debitos,
    c2.cpagodeservicios as lag1_cpagoservicios,
    c2.ctransferencias_recibidas as lag1_ctransferencias_in,
    c2.ctransferencias_emitidas as lag1_transferencias_out,
    c2.cextraccion_autoservicio as lag1_cextraccion,
    c2.ccheques_emitidos_rechazados as lag1_cheques_rechazados,
    c2.chomebanking_transacciones as lag1_hbtransacciones,
    c2.Master_delinquency as lag1_mdelinquency,
    c2.Visa_delinquency as lag1_vdelinquency,
    c2.Master_msaldototal as lag1_msaldototalmaster,
    c2.Visa_msaldototal as lag1_msaldototalvisa,
    c2.Master_mconsumospesos as lag1_masterconsumopesos,
    c2.Visa_mconsumospesos as lag1_visaconsumopesos,
    c2.Master_mpagado as lag1_master_mpagado,
    c2.Visa_mpagado as lag1_visa_mpagado,
    c2.Master_mconsumototal as lag1_mastermconsumo,
    c2.Visa_mconsumototal as lag1_visamconsumo,
    c2.mcuentas_saldo_1 as lag1_mcuentas_saldo1,
    ifnull(c1.new_totalpayroll, 0) - ifnull(c2.new_totalpayroll, 0) as diff_totalpayroll,
    ifnull(c1.ratio_avrgtransaction, 0) - ifnull(c2.ratio_avrgtransaction, 0) as diff_avrgtransaction,
    ifnull(c1.new_montransacciones, 0) - ifnull(c2.new_montransacciones, 0) as diff_montransacciones,
    ifnull(c1.new_cantransacciones, 0) - ifnull(c2.new_cantransacciones, 0) as diff_cantransacciones,
    ifnull(c1.mrentabilidad, 0) - ifnull(c2.mrentabilidad, 0) as diff_mrentabilidad,
    ifnull(c1.mcuentas_saldo, 0) - ifnull(c2.mcuentas_saldo, 0) as diff_mcuentas_saldo,
    ifnull(c1.new_limitedecompra, 0) - ifnull(c2.new_limitedecompra, 0) as diff_limitedecompra,
    ifnull(c1.actividad, 0) - ifnull(c2.actividad, 0) as diff_actividad,
    ifnull(c1.ratio_consumocuenta, 0) - ifnull(c2.ratio_consumocuenta, 0) as diff_ratioconsumocuenta,
    ifnull(c1.Visa_mconsumototal,0) - ifnull(c2.Visa_mconsumototal, 0) as diff_visaconsumo,
    ifnull(c1.mpasivos_margen,0) - ifnull(c2.mpasivos_margen,0) as diff_pasivos,
    ifnull(c1.mactivos_margen,0) - ifnull(c2.mactivos_margen,0) as diff_activos,
    ifnull(c1.mcuenta_corriente,0) - ifnull(c2.mcuenta_corriente,0) as diff_mcuenta_corriente,
    ifnull(c1.mcaja_ahorro,0) - ifnull(c2.mcaja_ahorro,0) as diff_mcajaahorro,
    ifnull(c1.ctarjeta_debito_transacciones,0) - ifnull(c2.ctarjeta_debito_transacciones,0) as diff_cdebitotrans,
    ifnull(c1.ctarjeta_visa_transacciones,0) - ifnull(c2.ctarjeta_visa_transacciones,0) as diff_cisatrans,
    ifnull(c1.mtarjeta_visa_consumo,0) - ifnull(c2.mtarjeta_visa_consumo,0)as diff_mvisaconsumo,
    ifnull(c1.ctarjeta_master_transacciones,0) -  ifnull(c2.ctarjeta_master_transacciones,0) as diff_cmasterconsumo,
    ifnull(c1.mtarjeta_master_consumo,0) - ifnull(c2.mtarjeta_master_consumo,0) as diff_mmasterconsumo,
    ifnull(c1.mprestamos_personales,0) - ifnull(c2.mprestamos_personales,0) as diff_prestamos,
    ifnull(c1.mplazo_fijo_dolares,0) -  ifnull(c2.mplazo_fijo_dolares,0) as diff_mfijo_dolares,
    ifnull(c1.mplazo_fijo_pesos,0) -  ifnull(c2.mplazo_fijo_pesos,0) as diff_mfijo_dolares,
    ifnull(c1.ccuenta_debitos_automaticos,0) -  ifnull(c2.ccuenta_debitos_automaticos,0)as diff_ccuenta_debitos,
    ifnull(c1.mcuenta_debitos_automaticos,0) - ifnull(c2.mcuenta_debitos_automaticos,0) as diff_mcuenta_debitos,
    ifnull(c1.ctarjeta_visa_debitos_automaticos,0) - ifnull(c2.ctarjeta_visa_debitos_automaticos,0) as diff_visa_debitos,
    ifnull(c1.ctransferencias_recibidas,0) - ifnull(c2.ctransferencias_recibidas,0)as diff_ctransferencias_in,
    ifnull(c1.ctransferencias_emitidas,0) - ifnull(c2.ctransferencias_emitidas,0) as diff_transferencias_out,
    ifnull(c1.Master_msaldototal,0) -  ifnull(c2.Master_msaldototal,0) as diff_msaldototalmaster,
    ifnull(c1.Visa_msaldototal,0) - ifnull(c2.Visa_msaldototal,0) as diff_msaldototalvisa,
    ifnull(c1.Master_mpagado,0) - ifnull(c2.Master_mpagado,0) as diff_master_mpagado,
    ifnull(c1.Visa_mpagado,0) -  ifnull(c2.Visa_mpagado,0) as diff_visa_mpagado,
    ifnull(c1.Master_mconsumototal,0) - ifnull(c2.Master_mconsumototal,0) as diff_mastermconsumo,
    ifnull(c1.Visa_mconsumototal,0) - ifnull(c2.Visa_mconsumototal,0)as diff_visamconsumo,
    ifnull(c1.mcuentas_saldo_1,0) -ifnull(c2.mcuentas_saldo_1,0) as diff_mcuentas_saldo1,

from competencia_01 c1
left join competencia_01 c2
    on c1.numero_de_cliente = c2.numero_de_cliente
    and c1.foto_mes = c2.foto_mes + 1;


In [ ]:
%%sql
create or replace table competencia_01_lag as
select
    c1.*,
    c2.new_totalpayroll as lag2_new_totalpayroll,
    c2.new_deudor as lag2_new_deudor,
    c2.ratio_avrgtransaction as lag2_ratio_avrgtransaction,
    c2.ratio_consumocuenta as lag2_ratio_consumocuenta,
    c2.new_montransacciones as lag2_new_montransacciones,
    c2.new_cantransacciones as lag2_new_cantransacciones,
    c2.mrentabilidad as lag2_mrentabilidad,
    c2.mcuentas_saldo as lag2_mcuentas_saldo,
    c2.new_limitedecompra as lag2_new_limitedecompra,
    c2.new_total_gain as lag2_new_total_gain,
    c2.new_total_comiss as lag2_new_total_comiss,
    c2.mcomisiones as lag2_mcomisiones,
    c2.new_total_discounts as lag2_new_total_discounts,
    c2.actividad as lag2_actividad,
    c2.mpasivos_margen as lag2_pasivos,
    c2.mactivos_margen as lag2_activos,
    c2.tcuentas as lag2_tcuentas,
    c2.mcuenta_corriente as lag2_mcuenta_corriente,
    c2.mcuenta_corriente_adicional as lag2_mcuentaadicional,
    c2.mcaja_ahorro as lag2_mcajaahorro,
    c2.ctarjeta_debito_transacciones as lag2_cdebitotrans,
    c2.ctarjeta_visa_transacciones as lag2_cisatrans,
    c2.mtarjeta_visa_consumo as lag2_mvisaconsumo,
    c2.ctarjeta_master_transacciones as lag2_cmasterconsumo,
    c2.mtarjeta_master_consumo as lag2_mmasterconsumo,
    c2.mprestamos_personales as lag2_prestamos,
    c2.mplazo_fijo_dolares as lag2_mfijo_dolares,
    c2.mplazo_fijo_pesos as lag2_mfijo_pesos,
    c2.ccuenta_debitos_automaticos as lag2_ccuenta_debitos,
    c2.mcuenta_debitos_automaticos as lag2_mcuenta_debitos,
    c2.ctarjeta_visa_debitos_automaticos as lag2_visa_debitos,
    c2.cpagodeservicios as lag2_cpagoservicios,
    c2.ctransferencias_recibidas as lag2_ctransferencias_in,
    c2.ctransferencias_emitidas as lag2_transferencias_out,
    c2.cextraccion_autoservicio as lag2_cextraccion,
    c2.ccheques_emitidos_rechazados as lag2_cheques_rechazados,
    c2.chomebanking_transacciones as lag2_hbtransacciones,
    c2.Master_delinquency as lag2_mdelinquency,
    c2.Visa_delinquency as lag2_vdelinquency,
    c2.Master_msaldototal as lag2_msaldototalmaster,
    c2.Visa_msaldototal as lag2_msaldototalvisa,
    c2.Master_mconsumospesos as lag2_masterconsumopesos,
    c2.Visa_mconsumospesos as lag2_visaconsumopesos,
    c2.Master_mpagado as lag2_master_mpagado,
    c2.Visa_mpagado as lag2_visa_mpagado,
    c2.Master_mconsumototal as lag2_mastermconsumo,
    c2.Visa_mconsumototal as lag2_visamconsumo,
    c2.mcuentas_saldo_1 as lag2_mcuentas_saldo1,

from competencia_01_lag c1
left join competencia_01_lag c2
    on c1.numero_de_cliente = c2.numero_de_cliente
    and c1.foto_mes = c2.foto_mes + 2;

In [ ]:
%%sql COPY competencia_01_lag TO '{{dataset_path}}competencia_01_base02.csv' (FORMAT CSV, HEADER)

# Optimización del modelo

In [ ]:
base_path = '/content/drive/MyDrive/EyF/'
dataset_path = base_path
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_01_base02.csv'

ganancia_acierto = 780000
costo_estimulo = 20000

#modelo
mes_train = [202101, 202102]
mes_test = 202103


# agregue sus semillas
semillas = [1009, 1223, 1427, 1699, 2029]

data = pd.read_csv(dataset_path + dataset_file)

In [ ]:
data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [ ]:
data['clase_binaria1'] = 0
#data['clase_binaria2'] = 0
#data['clase_binaria1'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)
data['clase_binaria1'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1)

In [ ]:
#train_data = data[data['foto_mes'] == mes_train]
train_data = data[data['foto_mes'].isin(mes_train)].copy()
test_data = data[data['foto_mes'] == mes_test]

X_train = train_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1'], axis=1) #dropear tambien num cliente y foto mes?
y_train_binaria1 = train_data['clase_binaria1']
#y_train_binaria2 = train_data['clase_binaria2']
w_train = train_data['clase_peso']

X_test = test_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1'], axis=1)
y_test_binaria1 = test_data['clase_binaria1']
y_test_class = test_data['clase_ternaria']
w_test = test_data['clase_peso']

In [ ]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

# Parámetros del modelos.
params = {
    'objective': 'binary',
    'metric': 'gan_eval',
    'boosting_type': 'gbdt',
    'max_bin': 31,
    'num_leaves': 31,
    'learning_rate': 0.01,
    'feature_fraction': 0.3,
    'bagging_fraction': 0.7,
    'verbose': 0
}

In [ ]:
train_data1 = lgb.Dataset(X_train, label=y_train_binaria1, weight=w_train)

In [ ]:
def objective(trial):

    num_leaves = trial.suggest_int('num_leaves', 6, 800)
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.5) # mas bajo, más iteraciones necesita
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 1, 2000)
    feature_fraction = trial.suggest_float('feature_fraction', 0.1, 1.0)
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.1, 1.0)
    num_iterations = trial.suggest_int('num_iterations', 10, 150)
    lambda_l1 = trial.suggest_float('lambda_l1', 0.0, 10.0)
    #boosting_type = trial.suggest_categorical('boosting_type', ['gbdt', 'dart', 'goss'])

    params = {
        'objective': 'binary',
        'metric': 'custom',
        #'metric': 'auc',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'lambda_l1': lambda_l1,
        #'boosting_type': boosting_type,
        'seed': semillas[2], #cambiar semilla cada aprox 20 trials
        'verbose': -1
    }
    train_data = lgb.Dataset(X_train,
                              label=y_train_binaria1, # eligir la clase
                              weight=w_train)
    cv_results = lgb.cv(
        params,
        train_data,
        num_boost_round=160,
        callbacks=[lgb.early_stopping(stopping_rounds=int(50 + 5 / learning_rate))],
        feval=lgb_gan_eval,
        stratified=True,
        nfold=5,
        seed=semillas[2]
    )
    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

    # Guardamos cual es la mejor iteración del modelo
    trial.set_user_attr("best_iter", best_iter)

    return max_gan * 5


storage_name = "sqlite:///" + db_path + "optimization_lgbm007.db"
study_name = "comp_1_lgbm_trial007"

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

In [ ]:
study.optimize(objective, n_trials=160) #cambiar semillas cada 20 aprox

In [ ]:
study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
    sampler=optuna.samplers.RandomSampler(seed=42),
)

In [ ]:
study.optimize(objective, n_trials=20)

# Importante, en caso de replicar.

Estos fueron los resultados de optuna durante el proceso anterior: {'objective': 'binary', 'boosting_type': 'gbdt', 'first_metric_only': True, 'boost_from_average': True, 'feature_pre_filter': False, 'max_bin': 31, 'num_leaves': 591, 'learning_rate': 0.08460705217230861, 'min_data_in_leaf': 187, 'feature_fraction': 0.2793671138905695, 'bagging_fraction': 0.6298945329115464, 'seed': 2029, 'verbose': 0}

In [ ]:
best_iter = study.best_trial.user_attrs["best_iter"]
print(f"Mejor cantidad de árboles para el mejor model {best_iter}")
params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'max_bin': 31,
    'num_leaves': study.best_trial.params['num_leaves'],
    'learning_rate': study.best_trial.params['learning_rate'],
    'min_data_in_leaf': study.best_trial.params['min_data_in_leaf'],
    'feature_fraction': study.best_trial.params['feature_fraction'],
    'bagging_fraction': study.best_trial.params['bagging_fraction'],
    'seed': semillas[0],#se guardan 5 modelos cambiando solo la semilla
    'verbose': 0
}

train_data = lgb.Dataset(X_train,
                          label=y_train_binaria1,
                          weight=w_train)

model = lgb.train(params,
                  train_data,
                  num_boost_round=best_iter)


In [ ]:
 model.save_model(modelos_path + 'lgb_comp1_006_sem0.txt') #se guarda el mismo modelo cambiando solo la semilla

In [ ]:
best_iter = study.best_trial.user_attrs["best_iter"]
print(f"Mejor cantidad de árboles para el mejor model {best_iter}")
params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'max_bin': 31,
    'num_leaves': study.best_trial.params['num_leaves'],
    'learning_rate': study.best_trial.params['learning_rate'],
    'min_data_in_leaf': study.best_trial.params['min_data_in_leaf'],
    'feature_fraction': study.best_trial.params['feature_fraction'],
    'bagging_fraction': study.best_trial.params['bagging_fraction'],
    'seed': semillas[1],#se guardan 5 modelos cambiando solo la semilla
    'verbose': 0
}

train_data = lgb.Dataset(X_train,
                          label=y_train_binaria1,
                          weight=w_train)

model = lgb.train(params,
                  train_data,
                  num_boost_round=best_iter)

In [ ]:
model.save_model(modelos_path + 'lgb_comp1_006_sem1.txt') #se guarda el mismo modelo cambiando solo la semilla

In [ ]:
best_iter = study.best_trial.user_attrs["best_iter"]
print(f"Mejor cantidad de árboles para el mejor model {best_iter}")
params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'max_bin': 31,
    'num_leaves': study.best_trial.params['num_leaves'],
    'learning_rate': study.best_trial.params['learning_rate'],
    'min_data_in_leaf': study.best_trial.params['min_data_in_leaf'],
    'feature_fraction': study.best_trial.params['feature_fraction'],
    'bagging_fraction': study.best_trial.params['bagging_fraction'],
    'seed': semillas[2],#se guardan 5 modelos cambiando solo la semilla
    'verbose': 0
}

train_data = lgb.Dataset(X_train,
                          label=y_train_binaria1,
                          weight=w_train)

model = lgb.train(params,
                  train_data,
                  num_boost_round=best_iter)

In [ ]:
model.save_model(modelos_path + 'lgb_comp1_006_sem2.txt') #se guarda el mismo modelo cambiando solo la semilla

In [ ]:
best_iter = study.best_trial.user_attrs["best_iter"]
print(f"Mejor cantidad de árboles para el mejor model {best_iter}")
params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'max_bin': 31,
    'num_leaves': study.best_trial.params['num_leaves'],
    'learning_rate': study.best_trial.params['learning_rate'],
    'min_data_in_leaf': study.best_trial.params['min_data_in_leaf'],
    'feature_fraction': study.best_trial.params['feature_fraction'],
    'bagging_fraction': study.best_trial.params['bagging_fraction'],
    'seed': semillas[3],#se guardan 5 modelos cambiando solo la semilla
    'verbose': 0
}

train_data = lgb.Dataset(X_train,
                          label=y_train_binaria1,
                          weight=w_train)

model = lgb.train(params,
                  train_data,
                  num_boost_round=best_iter)

In [ ]:
model.save_model(modelos_path + 'lgb_comp1_006_sem3.txt') #se guarda el mismo modelo cambiando solo la semilla

In [ ]:
best_iter = study.best_trial.user_attrs["best_iter"]
print(f"Mejor cantidad de árboles para el mejor model {best_iter}")
params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'max_bin': 31,
    'num_leaves': study.best_trial.params['num_leaves'],
    'learning_rate': study.best_trial.params['learning_rate'],
    'min_data_in_leaf': study.best_trial.params['min_data_in_leaf'],
    'feature_fraction': study.best_trial.params['feature_fraction'],
    'bagging_fraction': study.best_trial.params['bagging_fraction'],
    'seed': semillas[4],#se guardan 5 modelos cambiando solo la semilla
    'verbose': 0
}

train_data = lgb.Dataset(X_train,
                          label=y_train_binaria1,
                          weight=w_train)

model = lgb.train(params,
                  train_data,
                  num_boost_round=best_iter)

In [ ]:
model.save_model(modelos_path + 'lgb_comp1_006_sem4.txt') #se guarda el mismo modelo cambiando solo la semilla

#Definición del umbral

In [ ]:
base_path = '/content/drive/MyDrive/EyF/'
dataset_path = base_path
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file1 = 'competencia_01_base02.csv'
#dataset_file2 = 'competencia_01_base01.csv'

ganancia_acierto = 780000
costo_estimulo = 20000

#modelo
#mes_train = 202103
#mes_test = 202104

#kaggle
mes_train = [202103,202102]
mes_test = 202104

# agregue sus semillas
semillas = [1009, 1223, 1427, 1699, 2029]


In [ ]:
data = pd.read_csv(dataset_path + dataset_file1)

In [ ]:
data['clase_peso'] = 1.0

#data2.loc[data2['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [ ]:
data['clase_binaria1'] = 0
#data['clase_binaria2'] = 0
#data['clase_binaria1'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)
data['clase_binaria1'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1)

In [ ]:
#train_data = data[data['foto_mes'] == mes_train]
train_data1 = data[data['foto_mes'].isin(mes_train)].copy()
test_data = data[data['foto_mes'] == mes_test]

X_train1 = train_data1.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1'], axis=1)
y_train_binaria1 = train_data1['clase_binaria1']
#y_train_binaria2 = train_data['clase_binaria2']
w_train1 = train_data1['clase_peso']


X_test = test_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1'], axis=1)
y_test_binaria1 = test_data['clase_binaria1']
y_test_class = test_data['clase_ternaria']
w_test = test_data['clase_peso']

In [ ]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

# Parámetros del modelos.
params = {
    'objective': 'binary',
    'metric': 'gan_eval',
    'boosting_type': 'gbdt',
    'max_bin': 31,
    'num_leaves': 31,
    'learning_rate': 0.01,
    'feature_fraction': 0.3,
    'bagging_fraction': 0.7,
    'verbose': 0
}

In [ ]:
train_data1 = lgb.Dataset(X_train1, label=y_train_binaria1, weight=w_train1)

In [ ]:

model1 = lgb.Booster(model_file=modelos_path + 'lgb_comp1_006_sem0.txt')
model2 = lgb.Booster(model_file=modelos_path + 'lgb_comp1_006_sem1.txt')
model3 = lgb.Booster(model_file=modelos_path + 'lgb_comp1_006_sem2.txt')
model4 = lgb.Booster(model_file=modelos_path + 'lgb_comp1_006_sem3.txt')
model5 = lgb.Booster(model_file=modelos_path + 'lgb_comp1_006_sem4.txt')

In [ ]:
y_pred_lgm1 = model1.predict(X_test)
y_pred_lgm2 = model2.predict(X_test)
y_pred_lgm3 = model3.predict(X_test)
y_pred_lgm4 = model4.predict(X_test)
y_pred_lgm5 = model5.predict(X_test)

In [ ]:

preds = (y_pred_lgm1 + y_pred_lgm2 + y_pred_lgm3 +y_pred_lgm4 + y_pred_lgm5)/5

In [ ]:
def ganancia_por_topn(y_pred, y_true, prop=1):
    y_pred = np.asarray(y_pred)
    y_true = np.asarray(y_true)  # <- esto evita el KeyError

    orden = np.argsort(-y_pred)
    y_true_ordenado = y_true[orden]

    ganancias_ind = np.where(y_true_ordenado == 1, ganancia_acierto, -costo_estimulo)
    ganancia_acum = np.cumsum(ganancias_ind) / prop

    n_opt = np.argmax(ganancia_acum)
    ganancia_max = ganancia_acum[n_opt]

    return ganancia_acum, n_opt + 1, ganancia_max

# Ejemplo con tus datos
ganancia_acum, n_opt, ganancia_max = ganancia_por_topn(y_pred_lgm1, y_test_binaria1)

# Gráfico
plt.figure(figsize=(8,5))
plt.plot(range(1, len(ganancia_acum)+1), ganancia_acum)
plt.axvline(n_opt, color='red', linestyle='--', label=f'Óptimo N={n_opt}')
plt.title("Ganancia acumulada según cantidad de envíos")
plt.xlabel("Cantidad de envíos (top N)")
plt.ylabel("Ganancia acumulada")
plt.legend()
plt.grid(True)
plt.show()

print(f"➡️ Mejor cantidad de envíos: {n_opt}")
print(f"➡️ Ganancia máxima: {ganancia_max:,.0f}")

#Predicción

In [ ]:
base_path = '/content/drive/MyDrive/EyF/'
dataset_path = base_path
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file1 = 'competencia_01_base02.csv'
#dataset_file2 = 'competencia_01_base01.csv'

ganancia_acierto = 780000
costo_estimulo = 20000

#modelo
#mes_train = 202103
#mes_test = 202104

#kaggle
mes_train = [202104,202103,202102,202101]
mes_test = 202106

# agregue sus semillas
semillas = [1009, 1223, 1427, 1699, 2029]

#data1 = pd.read_csv(dataset_path + dataset_file1)

In [ ]:
data2 = pd.read_csv(dataset_path + dataset_file1)

In [ ]:
data2['clase_peso'] = 1.0

data2.loc[data2['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data2.loc[data2['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [ ]:
data2['clase_binaria1'] = 0
#data['clase_binaria2'] = 0
#data['clase_binaria1'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)
data2['clase_binaria1'] = np.where(data2['clase_ternaria'] == 'CONTINUA', 0, 1)

In [ ]:
#train_data = data[data['foto_mes'] == mes_train]
train_data2 = data2[data2['foto_mes'].isin(mes_train)].copy()
X_futuro2 = data2[data2['foto_mes'] == mes_test]

X_train2 = train_data2.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1'], axis=1)
y_train_binaria2 = train_data2['clase_binaria1']
#y_train_binaria2 = train_data['clase_binaria2']
w_train2 = train_data2['clase_peso']


In [ ]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

# Parámetros del modelos.
params = {
    'objective': 'binary',
    'metric': 'gan_eval',
    'boosting_type': 'gbdt',
    'max_bin': 31,
    'num_leaves': 31,
    'learning_rate': 0.01,
    'feature_fraction': 0.3,
    'bagging_fraction': 0.7,
    'verbose': 0
}

In [ ]:
train_data2 = lgb.Dataset(X_train2, label=y_train_binaria2, weight=w_train2)

In [ ]:

model1 = lgb.Booster(model_file=modelos_path + 'lgb_comp1_006_sem0.txt')
model2 = lgb.Booster(model_file=modelos_path + 'lgb_comp1_006_sem1.txt')
model3 = lgb.Booster(model_file=modelos_path + 'lgb_comp1_006_sem2.txt')
model4 = lgb.Booster(model_file=modelos_path + 'lgb_comp1_006_sem3.txt')
model5 = lgb.Booster(model_file=modelos_path + 'lgb_comp1_006_sem4.txt')

In [ ]:
X_futuro2 = X_futuro2.drop(['clase_ternaria','clase_binaria1','clase_peso'], axis=1)
y_pred_lgm1 = model1.predict(X_futuro2)
y_pred_lgm2 = model2.predict(X_futuro2)
y_pred_lgm3 = model3.predict(X_futuro2)
y_pred_lgm4 = model4.predict(X_futuro2)
y_pred_lgm5 = model5.predict(X_futuro2)


In [ ]:
y_pred_lgm = (y_pred_lgm1 + y_pred_lgm2 + y_pred_lgm3 + y_pred_lgm4 + y_pred_lgm5)/5

In [ ]:

N = 15436  # cantidad de envíos deseada, calculada en la sección anterior

# Ordenamos por probabilidad descendente
orden = np.argsort(-y_pred_lgm)

# Inicializamos todo en 0 y marcamos 1 para los top N
preds = np.zeros_like(y_pred_lgm, dtype=int)
preds[orden[:N]] = 1

In [ ]:
from google.colab import files
from datetime import datetime

fecha = datetime.today().strftime('%Y-%m-%d')  # Genera la fecha en formato YYYY-MM-DD
nombre_archivo = f"solucion_lgb{fecha}.csv"

ids = X_futuro2['numero_de_cliente']

salida = pd.DataFrame({
    'numero_de_cliente': ids,
    'Predicted': preds
})

#print(salida)

# Guardar a CSV
salida.to_csv(nombre_archivo, index=False)
files.download(nombre_archivo)